# CloudFront 日志 ETL
这个试验将展示如果对 CloudFront 日志进行 ELT 操作，在开始前我们先从 [CloudFront Log Format](https://docs.aws.amazon.com/AmazonCloudFront/latest/DeveloperGuide/AccessLogs.html#LogFileFormat) 中了解日志的字段的含义。

该 ELT 实验有两个目的：
* 为每一条记录增加**国家**和**城市**信息。该任务可以通过查询IP数据库完成，此处我们使用 [IP2Location Lite](https://lite.ip2location.com/) 数据库。 
* 为每一条记录增加**设备品牌**, **操作系统**, **操作系统版本**信息。该任务可以通过解析user-agent字段完成，此处我们使用第三方Python库[user_agents](https://github.com/selwin/python-user-agents)。


### 1. 对源数据进行爬虫分析

我们下载了一部分 CloudFront 日志，并且以 csv 文件的格式保存到了S3中。

    s3://joeshi/customer/hanhui-dongyou/cloudfront/
        
爬虫会自动帮助我们分析数据结构，我们将结果保存到 Glue Date Catetegory 的数据库中，起名 `cloudfront_log`. 数据库中包含一张 `cloudfront` 的表.

此处我们可以通过 **Athena** 预览数据.

### 2. 准备实验

我们需要在实验开始前加载如下内容：

1. 导入 **user_agents**， **ip2location** 第三方 Python 库
2. 导入 **pyspark** 中使用到的数据类型和 UDF 
3. 创建一个 **GlueContext**

In [1]:
import boto3
from ip2location import IP2Location
from user_agents import parse

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

from pyspark.sql.types import StringType, DoubleType, StructType, StructField, Row
from pyspark.sql.functions import udf

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1548337377007_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### 3. 下载 IP2Location

将 IP2Location Lite 数据库下载并加载到内存。 数据库的文件位置位于

    s3://joeshi/PoC/Glue/artifacts/IP2LOCATION-LITE-DB5.IPV6.BIN
    
我们通过 AWS Python SDK 下载，并保存到 `/tmp` 目录下


In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('joeshi')
localPath = '/tmp/DB.BIN'
bucket.download_file('PoC/Glue/artifacts/IP2LOCATION-LITE-DB5.IPV6.BIN', localPath)
database = IP2Location.IP2Location()
database.open(localPath)

### 4. 创建解析user-agent字段的UDF

在 CloudFront 日志中，`cs-user-agent` 字段表示 **user-agent**, 如下示例

    Dalvik/2.1.0%2520(Linux;%2520U;%2520Android%25205.0;%2520Lenovo%2520K50-T5%2520Build/LRX21M)
   
在 CloudFront 日志中 `%2520` 表示空格，因此我们需要将其替换，然后作为`user_agents`库的输入。

In [3]:
def chop_ua(ua_string):
    user_agent = parse(ua_string.replace("%2520", " "))
    print(str(user_agent))
    return Row("ua_os_family", "ua_os_version", "ua_device_brand")(user_agent.os.family, user_agent.os.version_string, user_agent.device.brand)


ua_schema = StructType([
    StructField("ua_os_family", StringType(), False),
    StructField("ua_os_version", StringType(), False),
    StructField("ua_device_brand", StringType(), False)
])

chop_ua_udf = udf(chop_ua, ua_schema)

### 5. 创建将 IP String 转化成 Numeric 类型的UDF

使用 IP2Location Lite 数据库，根据 `c-ip` 字段查询该 IP 所在的国家、城市、经纬度信息。

In [4]:
def chop_c_ip(c_ip):
    rec = database.get_all(c_ip)
    return Row("country_short",
               "country_long",
               "city",
               "latitude",
               "longitude"
               )(rec.country_short,
                 rec.country_long,
                 rec.city,
                 rec.latitude,
                 rec.longitude)


# 如果是某一个 Field 需要转化成多个 Column，使用 StructField来实现
c_ip_schema = StructType([
    StructField("country_short", StringType(), False),
    StructField("country_long", StringType(), False),
    StructField("city", StringType(), False),
    StructField("latitude", DoubleType(), False),
    StructField("longitude", DoubleType(), False)
])

chop_c_ip_udf = udf(chop_c_ip, c_ip_schema)

### 6. 检查 Glue Crawler 爬虫分析的表结构

将 Glue 中的 `cloudfront` 表的表结果打印出来，并检查结构是否正确。

创建一个Glue **DynamicFrame**


In [5]:
cf_logs = glueContext.create_dynamic_frame.from_catalog(database="cloudfront_log", table_name="cloudfront")
print "Counts: ", cf_logs.count()
cf_logs.printSchema()

Counts:  171866
root
|-- date: string
|-- time: string
|-- x_edge_location: string
|-- sc_bytes: long
|-- c_ip: string
|-- cs_method: string
|-- cs_host: string
|-- cs_uri_stem: string
|-- sc_status: long
|-- cs_referer: string
|-- cs_user_agent: string
|-- cs_uri_query: string
|-- cs_cookie: string
|-- x_edge_result_type: string
|-- x_edge_request_id: string
|-- x_host_header: string
|-- cs_protocol: string
|-- cs_bytes: long
|-- time_taken: double
|-- x_forwarded_for: string
|-- ssl_protocol: string
|-- ssl_cipher: string
|-- x_edge_response_result_type: string
|-- cs_protocol_version: string
|-- mbps: string

### 7. 为记录增加 user agent 和地址位置信息

将 DynamicFrame 转化成 pyspark 中的 DataFrame, 并且通过 **withColumn** 和 **udf** 增加列。

打印检查新生成的 schema。

In [6]:
cf_logs_df = cf_logs.toDF()
cf_logs_df = cf_logs_df.withColumn("ua", chop_ua_udf(cf_logs_df.cs_user_agent))\
    .withColumn("c_ip_rec", chop_c_ip_udf(cf_logs_df.c_ip))
cf_logs_df = cf_logs_df.select("x_edge_location", "sc_bytes", "c_ip", "cs_uri_stem", "cs_user_agent", "x_edge_result_type",
                       "time_taken", "x_edge_response_result_type", "ua.*", "c_ip_rec.*")
cf_logs_df.printSchema()

root
 |-- x_edge_location: string (nullable = true)
 |-- sc_bytes: long (nullable = true)
 |-- c_ip: string (nullable = true)
 |-- cs_uri_stem: string (nullable = true)
 |-- cs_user_agent: string (nullable = true)
 |-- x_edge_result_type: string (nullable = true)
 |-- time_taken: double (nullable = true)
 |-- x_edge_response_result_type: string (nullable = true)
 |-- ua_os_family: string (nullable = true)
 |-- ua_os_version: string (nullable = true)
 |-- ua_device_brand: string (nullable = true)
 |-- country_short: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

### 8. (可选)预览结果

显示前10条结果

In [ ]:
cf_logs_df.show(10);

### 9. 将清洗完的结果输出

将清洗完的结果输出到 `s3://joeshi/PoC/Glue/job/cloudfront_etl_test/`;

建议使用 **Parquet** 或者 **ORC** 作为输出格式，这种列式存储的文件更适合大数据的查询.

In [ ]:
cf_logs = DynamicFrame.fromDF(cf_logs_df, glueContext, "cloudfront_parquet")
glueContext.write_dynamic_frame.from_options(frame = cf_logs,
                                             connection_type = "s3",
                                             connection_options = {"path": "s3://joeshi/PoC/Glue/job/cloudfront_etl_test/"},
                                             format = "parquet")

实验结束